In [3]:
pip install pennylane

     ---------------------------------------- 1.5/1.5 MB 651.0 kB/s eta 0:00:00
     -------------------------------------- 49.3/49.3 kB 830.0 kB/s eta 0:00:00
     ---------------------------------------- 49.9/49.9 kB 1.2 MB/s eta 0:00:00
     ---------------------------------------- 1.4/1.4 MB 634.3 kB/s eta 0:00:00
     ---------------------------------------- 4.2/4.2 MB 591.2 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pennylane as qml 
from pennylane import numpy as np
import matplotlib.pyplot as plt

In [102]:
m = 2
n = 2*m
def test_prep(m):
    for i in range(2*m):
        qml.Hadamard(i)
#    return [qml.expval(qml.PauliZ(i)) for i in range(2*m)]
#drawer = qml.draw(test_prep)
#print(drawer(2))
#test_prep(m)

In [103]:
#TODO
#Make angles equal in the ith subsystem of registers 1 and 2
#param needs to be specifiable with m angles
def ansatz(param, m):
    for i in range(2*m):
        qml.RY(param[i], wires = i)
        qml.RX(np.pi/2, wires = i)
#drawer = qml.draw(ansatz)
#print(drawer)

In [104]:
def cost_fun(m):
    for i in range(m):
        qml.CNOT(wires = [i,i+m])
#drawer = qml.draw(cost_fun)
#print(drawer(2))

In [134]:
s = 1000
m = 2
n = 2*m
dev = qml.device("default.qubit", wires = n, shots = s)
@qml.qnode(dev, interface = "autograd")
def circuit(param, m):
    test_prep(m)
    ansatz(param, m)
    cost_fun(m)
    return qml.probs(wires = [i for i in range(2*m)])

In [138]:
def VQSD(param):
    return circuit(param, m)[0]

In [147]:
t = []
for i in range(n):
    t.append(0.0)
theta = np.array(t, requires_grad=True)
print(theta)

[0. 0. 0. 0.]


In [148]:
diag = VQSD(theta)
print(diag)

0.049


In [79]:
# store the values of the circuit parameter
angle = [theta]
#print(angle)
#len(angle)

[tensor([0., 0., 0., 0.], requires_grad=True)]


1

In [83]:
opt = qml.GradientDescentOptimizer(stepsize=0.4)
max_iterations = 100
conv_tol = 1e-06

for n in range(max_iterations):
    theta, prev_diag = opt.step_and_cost(VQSD, theta)

    diag.append(VQSD(theta))
    angle.append(theta)

    conv = np.abs(energy[-1] - prev_diag)

    if n % 2 == 0:
        print(f"Step = {n},  Energy = {diag[-1]:.8f} Ha")

    if conv <= conv_tol:
        break

print("\n" f"Final value of the ground-state energy = {energy[-1]:.8f} Ha")
print("\n" f"Optimal value of the circuit parameter = {angle[-1]:.4f}")

TypeError: Grad only applies to real scalar-output functions. Try jacobian, elementwise_grad or holomorphic_grad.